In [1]:
import io

import netaddr

import pandas as pd

import aiohttp

from rpki_analysis.delegated_stats import read_delegated_stats

In [2]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://ftp.ripe.net/pub/stats/ripencc/nro-stats/latest/nro-delegated-stats') as resp:
        df = read_delegated_stats(io.StringIO(await resp.text()))


In [3]:
list(df.sample(1).resource)[0]

IPRange('195.210.9.0', '195.210.9.255')

In [4]:
# Calculate cover of VRPs
def join_resources(df: pd.DataFrame) -> pd.DataFrame:
    """Join the resources of a group"""
    return pd.DataFrame([{
        "resources": netaddr.IPSet(df[df.afi != "asn"].resource),
    }])


In [5]:
df_resources = df.groupby(['opaque_id', 'country', 'afi', 'rir']).apply(join_resources)

In [6]:
df_resources['resource_size'] = df_resources.resources.map(lambda x: x.size)

In [7]:
df_resources[['resource_size']].sample(1)


,,,,,resource_size
opaque_id,country,afi,rir,,
e65129e7-4bde-4836-bc8f-f4c68c897922,NL,ipv6,ripencc,0,79228162514264337593543950336


In [8]:
def majority_size(df: pd.DataFrame) -> float:
    sizes = list(df.resource_size)
    sizes.sort(reverse=True)
    total_sizes = sum(sizes)
    if total_sizes > 0:
        return pd.DataFrame([{"covered_fraction": sizes[0]/sum(sizes), "total_resources": total_sizes }])
    return pd.DataFrame([{"covered_fraction": 0, "total_resources": 0 }])
    
df_fraction_majority = df_resources[['resource_size']].reset_index().groupby(['opaque_id', 'afi', 'rir']).apply(majority_size)

In [9]:
df_fraction_majority

covered_fraction  \
opaque_id                            afi  rir                           
00004a29a9bbf62af6b8e5b91b760abc     asn  arin    0               0.0   
                                     ipv4 arin    0               1.0   
00005b39-e46c-45d2-90c9-2a9255d76e9c asn  ripencc 0               0.0   
                                     ipv4 ripencc 0               1.0   
000076699efb0f677b8a77052cacd0eb     asn  arin    0               0.0   
...                                                               ...   
lacnic                               ipv4 lacnic  0               1.0   
                                     ipv6 lacnic  0               1.0   
ripencc                              asn  ripencc 0               0.0   
                                     ipv4 ripencc 0               1.0   
                                     ipv6 ripencc 0               1.0   

                                                                          total_resources  
opaque_id                            afi  rir                                              
00004a29a9bbf62af6b8e5b91b760abc     asn  arin    0                                     0  
                                     ipv4 arin    0                                   256  
00005b39-e46c-45d2-90c9-2a9255d76e9c asn  ripencc 0                                     0  
                                     ipv4 ripencc 0                                  1280  
000076699efb0f677b8a77052cacd0eb     asn  arin    0                                     0  
...                                                                                   ...  
lacnic                               ipv4 lacnic  0                                149504  
                                     ipv6 lacnic  0   81826073880159943374251909751767040  
ripencc                              asn  ripencc 0                                     0  
                                     ipv4 ripencc 0                                662024  
                                     ipv6 ripencc 0  154717146735345794200279388765814784  

[235279 rows x 2 columns]

In [10]:
df_ip = df_fraction_majority.reset_index()[df_fraction_majority.reset_index().afi != "asn"]

In [11]:
df_ip[(df_ip.covered_fraction < 1.0) & (df_ip.rir == "ripencc")].sort_values(["total_resources"])

,opaque_id,afi,rir,level_3,covered_fraction,total_resources
211017,d5ee3bd4-738f-4d5e-910d-f8a70bbf631b,ipv6,ripencc,0,0.999998,633826509039934315377526308864


In [12]:
df[df.opaque_id == "b06b370c-da4c-429b-ba92-322c81c9662d"].country.unique()

[], Categories (239, object): ['AD', 'AE', 'AF', 'AG', ..., 'ZZ', 'FK', 'MS', 'ST']

Check how much of the space in the country is by RIPE NCC

In [13]:
def rir_fraction(df: pd.DataFrame, rir="ripencc") -> float:
    rir_total = sum(df[df.rir == rir].resource_size)
    total = sum(df.resource_size)
    
    if total > 0:
        return pd.DataFrame([{rir: rir_total/total, f"{rir}_resources": rir_total, "total_resources": total }])
    return pd.DataFrame([{rir: 0, f"{rir}_resources": 0, "total_resources": 0 }])
    
df_fraction_majority = df_resources[['resource_size']] \
    .reset_index() \
    .groupby(['afi', 'country']) \
    .apply(rir_fraction) \
    .reset_index()

In [14]:
df_fraction_majority[df_fraction_majority.afi != "asn"]

,afi,country,level_2,ripencc,ripencc_resources,total_resources
239,ipv4,AD,0,1.000000,56320,56320
240,ipv4,AE,0,0.992701,4804736,4840064
241,ipv4,AF,0,0.008683,1536,176896
242,ipv4,AG,0,0.632102,113920,180224
243,ipv4,AI,0,0.026316,256,9728
...,...,...,...,...,...,...
703,ipv6,ZA,0,0.003812,1505335087771022414277335056384,394879562024382498413073942970368
704,ipv6,ZM,0,0.000000,0,633831344743212773894225133568
705,ipv6,ZW,0,0.000000,0,1029972157314534461861944885248
706,ipv6,ZZ,0,0.000455,154717146735345794200279388765814784,340247015785117294229624333556672102400


In [15]:
import os

df_fraction_majority[df_fraction_majority.afi != "asn"].to_excel(os.path.expanduser("~/Desktop/fraction-non-ripe-ncc-by-afi-country.xlsx")) 